# Solutions: Pre-Transformer Era Exercises

This notebook provides complete solutions to the exercises from **`01_pre_transformer_era.ipynb`**.

## What You'll Learn

Through these exercises, you'll deepen your understanding of:

1. **RNN Architecture** - How hyperparameters and activation functions affect behavior
2. **Vanishing Gradients** - Why this is an exponential problem and the narrow "Goldilocks zone"
3. **LSTM Gates** - How gates dynamically control information flow
4. **Conceptual Thinking** - Brainstorming solutions that lead to transformers
5. **Language Modeling** - Building a complete character-level model with backpropagation

## How to Use This Notebook

- **Self-contained**: All necessary code is included (SimpleRNN, LSTMCell, etc.)
- **Progressive**: Exercises build from simple to complex
- **Visual**: Extensive visualizations help you understand concepts
- **Educational**: Not just answers, but detailed explanations of "why"

## Prerequisites

Before starting, make sure you've completed:
- ✅ `00_introduction_and_setup.ipynb`
- ✅ `01_pre_transformer_era.ipynb`

Let's dive in! 🚀

---

# Setup

First, let's import all dependencies and set up our environment.

In [ ]:
# Import required libraries
import sys
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import HTML

# Add utilities to path
sys.path.append(os.path.abspath('../src'))
from utils import set_style, print_tensor_info

# Set visual style for consistency
set_style()

# Set random seed for reproducibility
np.random.seed(42)

print("✓ Setup complete! Libraries imported and random seed set.")

## Helper Functions and Classes

We'll copy the necessary classes from the main notebook so this notebook is self-contained.

In [ ]:
class SimpleRNN:
    """
    A minimal RNN implementation in NumPy.
    
    This implements the equations:
      h_t = activation(W_hh @ h_{t-1} + W_xh @ x_t + b_h)
      y_t = W_hy @ h_t + b_y
      
    Modified to support different activation functions (tanh, ReLU).
    """
    
    def __init__(self, input_size, hidden_size, output_size, activation='tanh'):
        """
        Initialize the RNN with random weights.
        
        Args:
            input_size: Dimension of input vectors
            hidden_size: Dimension of hidden state
            output_size: Dimension of output
            activation: 'tanh' or 'relu'
        """
        self.hidden_size = hidden_size
        self.activation = activation
        
        # Weight matrices
        self.W_xh = np.random.randn(input_size, hidden_size) * 0.01
        self.W_hh = np.random.randn(hidden_size, hidden_size) * 0.01
        self.W_hy = np.random.randn(hidden_size, output_size) * 0.01
        
        # Biases
        self.b_h = np.zeros((hidden_size,))
        self.b_y = np.zeros((output_size,))
    
    def _activate(self, x):
        """Apply activation function."""
        if self.activation == 'tanh':
            return np.tanh(x)
        elif self.activation == 'relu':
            return np.maximum(0, x)
        else:
            raise ValueError(f"Unknown activation: {self.activation}")
    
    def forward(self, inputs):
        """
        Forward pass through the RNN.
        
        Args:
            inputs: Sequence of inputs, shape (seq_len, input_size)
        
        Returns:
            outputs: Sequence of outputs, shape (seq_len, output_size)
            hidden_states: All hidden states, shape (seq_len, hidden_size)
        """
        seq_len = len(inputs)
        hidden_states = []
        outputs = []
        
        # Initial hidden state (all zeros)
        h = np.zeros(self.hidden_size)
        
        # Process sequence
        for t in range(seq_len):
            x_t = inputs[t]
            
            # Update hidden state
            hidden_contribution = self.W_hh.T @ h
            input_contribution = self.W_xh.T @ x_t
            combined = hidden_contribution + input_contribution + self.b_h
            h = self._activate(combined)
            
            # Compute output
            y = self.W_hy.T @ h + self.b_y
            
            hidden_states.append(h.copy())
            outputs.append(y)
        
        return np.array(outputs), np.array(hidden_states)

print("✓ SimpleRNN class loaded")

In [ ]:
def sigmoid(x):
    """Sigmoid activation function."""
    return 1 / (1 + np.exp(-np.clip(x, -500, 500)))  # Clip to prevent overflow

class LSTMCell:
    """
    Single LSTM cell (processes one timestep).
    
    Implements the full LSTM equations with forget, input, and output gates.
    """
    
    def __init__(self, input_size, hidden_size):
        """Initialize LSTM cell with random weights."""
        self.hidden_size = hidden_size
        combined_size = hidden_size + input_size
        
        # Weight matrices for gates and cell candidate
        self.W_f = np.random.randn(combined_size, hidden_size) * 0.01
        self.W_i = np.random.randn(combined_size, hidden_size) * 0.01
        self.W_C = np.random.randn(combined_size, hidden_size) * 0.01
        self.W_o = np.random.randn(combined_size, hidden_size) * 0.01
        
        # Biases
        self.b_f = np.zeros(hidden_size)
        self.b_i = np.zeros(hidden_size)
        self.b_C = np.zeros(hidden_size)
        self.b_o = np.zeros(hidden_size)
    
    def forward(self, x_t, h_prev, C_prev):
        """
        Forward pass for one timestep.
        
        Args:
            x_t: Input at time t
            h_prev: Previous hidden state
            C_prev: Previous cell state
        
        Returns:
            h_t: New hidden state
            C_t: New cell state
            (f_t, i_t, o_t): Gate values
        """
        # Concatenate previous hidden state and current input
        combined = np.concatenate([h_prev, x_t])
        
        # Forget gate
        f_t = sigmoid(combined @ self.W_f + self.b_f)
        
        # Input gate
        i_t = sigmoid(combined @ self.W_i + self.b_i)
        
        # Cell candidate
        C_tilde = np.tanh(combined @ self.W_C + self.b_C)
        
        # Update cell state (the key equation!)
        C_t = f_t * C_prev + i_t * C_tilde
        
        # Output gate
        o_t = sigmoid(combined @ self.W_o + self.b_o)
        
        # Compute hidden state
        h_t = o_t * np.tanh(C_t)
        
        return h_t, C_t, (f_t, i_t, o_t)

print("✓ LSTMCell class loaded")

In [ ]:
def simulate_gradient_flow(seq_len=50, weight_value=0.9):
    """
    Simulate how gradients change as they flow backward through time in an RNN.
    
    Args:
        seq_len: Length of sequence
        weight_value: Value of recurrent weight W_hh
    
    Returns:
        Array of gradient values at each timestep
    """
    gradient = 1.0
    gradients = [gradient]
    tanh_derivative = 0.5  # Approximate average
    
    for t in range(seq_len - 1):
        gradient = gradient * weight_value * tanh_derivative
        gradients.append(gradient)
    
    return np.array(gradients[::-1])

print("✓ Helper functions loaded")
print("\n" + "="*70)
print("All setup complete! Ready to solve exercises.")
print("="*70)

---

# Exercise 1: Experiment with RNN

## Original Exercise

> **Task**: Modify the `SimpleRNN` class and observe the effects.
>
> Try:
> - Change `hidden_size` to 2, 8, 16. How do the hidden state visualizations change?
> - Replace `tanh` with `ReLU` (`np.maximum(0, x)`). What happens? Why might this be bad?
> - Process a longer sequence (seq_len=20). How do hidden states evolve?

## Learning Objectives

- Understand how `hidden_size` affects the RNN's representational capacity
- See why activation functions matter (tanh vs ReLU)
- Observe how RNNs process longer sequences

Let's explore each part!

## Part A: Hidden Size Variations

The `hidden_size` parameter controls how many numbers the RNN uses to remember information. Let's see how different sizes affect the behavior.

In [ ]:
# Create RNNs with different hidden sizes
input_size = 3
output_size = 2
seq_len = 5
hidden_sizes = [2, 8, 16]

# Generate same input for fair comparison
np.random.seed(42)
inputs = np.random.randn(seq_len, input_size)

# Store results for each hidden size
results = {}

for hidden_size in hidden_sizes:
    rnn = SimpleRNN(input_size, hidden_size, output_size)
    outputs, hidden_states = rnn.forward(inputs)
    results[hidden_size] = hidden_states
    print(f"Hidden size {hidden_size:2d}: hidden_states shape = {hidden_states.shape}")

print("\n✓ Processed sequence with 3 different hidden sizes")

In [ ]:
# Visualize hidden states for all three sizes
fig, axes = plt.subplots(3, 2, figsize=(15, 12))

for idx, hidden_size in enumerate(hidden_sizes):
    hidden_states = results[hidden_size]
    
    # Left column: Line plot
    ax_line = axes[idx, 0]
    ax_line.plot(hidden_states, linewidth=2, marker='o', markersize=4)
    ax_line.set_xlabel('Timestep', fontsize=11)
    ax_line.set_ylabel('Hidden State Value', fontsize=11)
    ax_line.set_title(f'Hidden Size = {hidden_size} (Line Plot)', fontsize=12, fontweight='bold')
    ax_line.legend([f'h[{i}]' for i in range(hidden_size)], ncol=2, fontsize=8)
    ax_line.grid(True, alpha=0.3)
    
    # Right column: Heatmap
    ax_heat = axes[idx, 1]
    sns.heatmap(hidden_states.T, 
                cmap='coolwarm', 
                center=0,
                xticklabels=range(seq_len),
                yticklabels=[f'h[{i}]' for i in range(hidden_size)],
                cbar_kws={'label': 'Value'},
                annot=True if hidden_size <= 8 else False,  # Only annotate for smaller sizes
                fmt='.2f',
                ax=ax_heat)
    ax_heat.set_xlabel('Timestep', fontsize=11)
    ax_heat.set_ylabel('Hidden Dimension', fontsize=11)
    ax_heat.set_title(f'Hidden Size = {hidden_size} (Heatmap)', fontsize=12, fontweight='bold')

plt.tight_layout()
plt.show()

### Analysis: What Do We Observe?

**Key Observations**:

1. **Hidden Size = 2** (Small Capacity)
   - Only 2 dimensions to store information
   - Limited representational capacity
   - Good for simple patterns, but can't capture complex relationships

2. **Hidden Size = 8** (Medium Capacity)
   - 8 dimensions provide more flexibility
   - Can capture more nuanced patterns
   - Good balance between capacity and efficiency

3. **Hidden Size = 16** (Large Capacity)
   - 16 dimensions offer high representational power
   - Can model complex relationships
   - But: more parameters = more data needed for training

**Why This Matters**:
- **Too small**: RNN can't capture enough information → underfitting
- **Too large**: More parameters to train, slower, risk of overfitting
- **Real-world**: Modern RNNs often use 256-1024 hidden dimensions for complex tasks like language modeling

## Part B: ReLU vs tanh Activation

Activation functions are crucial in RNNs. Let's compare tanh (the standard choice) with ReLU (popular in feedforward networks).

In [ ]:
# Create two RNNs: one with tanh, one with ReLU
np.random.seed(42)
input_size = 3
hidden_size = 4
output_size = 2
seq_len = 10

# Generate input
inputs = np.random.randn(seq_len, input_size)

# Create RNNs
rnn_tanh = SimpleRNN(input_size, hidden_size, output_size, activation='tanh')
rnn_relu = SimpleRNN(input_size, hidden_size, output_size, activation='relu')

# Make them use the same weights for fair comparison
rnn_relu.W_xh = rnn_tanh.W_xh.copy()
rnn_relu.W_hh = rnn_tanh.W_hh.copy()
rnn_relu.W_hy = rnn_tanh.W_hy.copy()
rnn_relu.b_h = rnn_tanh.b_h.copy()
rnn_relu.b_y = rnn_tanh.b_y.copy()

# Forward pass
outputs_tanh, hidden_tanh = rnn_tanh.forward(inputs)
outputs_relu, hidden_relu = rnn_relu.forward(inputs)

print("✓ Processed sequence with both tanh and ReLU")
print(f"\ntanh hidden states range: [{hidden_tanh.min():.3f}, {hidden_tanh.max():.3f}]")
print(f"ReLU hidden states range: [{hidden_relu.min():.3f}, {hidden_relu.max():.3f}]")

In [ ]:
# Visualize comparison
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# tanh - Line plot
axes[0, 0].plot(hidden_tanh, linewidth=2, marker='o', markersize=4)
axes[0, 0].set_xlabel('Timestep')
axes[0, 0].set_ylabel('Hidden State Value')
axes[0, 0].set_title('tanh Activation (Line Plot)', fontweight='bold')
axes[0, 0].legend([f'h[{i}]' for i in range(hidden_size)], ncol=2)
axes[0, 0].grid(True, alpha=0.3)
axes[0, 0].axhline(y=0, color='k', linestyle='--', alpha=0.3)

# tanh - Heatmap
sns.heatmap(hidden_tanh.T, cmap='coolwarm', center=0, 
            xticklabels=range(seq_len), yticklabels=[f'h[{i}]' for i in range(hidden_size)],
            cbar_kws={'label': 'Value'}, annot=True, fmt='.2f', ax=axes[0, 1])
axes[0, 1].set_xlabel('Timestep')
axes[0, 1].set_ylabel('Hidden Dimension')
axes[0, 1].set_title('tanh Activation (Heatmap)', fontweight='bold')

# ReLU - Line plot
axes[1, 0].plot(hidden_relu, linewidth=2, marker='s', markersize=4)
axes[1, 0].set_xlabel('Timestep')
axes[1, 0].set_ylabel('Hidden State Value')
axes[1, 0].set_title('ReLU Activation (Line Plot)', fontweight='bold')
axes[1, 0].legend([f'h[{i}]' for i in range(hidden_size)], ncol=2)
axes[1, 0].grid(True, alpha=0.3)
axes[1, 0].axhline(y=0, color='k', linestyle='--', alpha=0.3)

# ReLU - Heatmap
sns.heatmap(hidden_relu.T, cmap='coolwarm', center=0,
            xticklabels=range(seq_len), yticklabels=[f'h[{i}]' for i in range(hidden_size)],
            cbar_kws={'label': 'Value'}, annot=True, fmt='.2f', ax=axes[1, 1])
axes[1, 1].set_xlabel('Timestep')
axes[1, 1].set_ylabel('Hidden Dimension')
axes[1, 1].set_title('ReLU Activation (Heatmap)', fontweight='bold')

plt.tight_layout()
plt.show()

### Analysis: Why ReLU is Problematic for RNNs

**Observations**:

1. **tanh** (Top row):
   - Values bounded in [-1, 1]
   - Smooth evolution over time
   - Stable behavior

2. **ReLU** (Bottom row):
   - Values can be 0 or positive (no negative values)
   - Some dimensions may "die" (stuck at 0)
   - Can lead to unbounded growth (exploding activations)

**Why ReLU is Bad for RNNs**:

1. **Dying ReLU Problem**:
   - If `W_hh @ h + W_xh @ x + b_h < 0`, then ReLU outputs 0
   - That dimension becomes "dead" and can never recover
   - You lose that dimension's capacity permanently

2. **Exploding Activations**:
   - ReLU has no upper bound (unlike tanh which saturates at 1)
   - Repeated multiplication: `h_t = ReLU(W @ h_{t-1} + ...)` can cause values to grow exponentially
   - This leads to numerical instability and NaN values

3. **No Negative Values**:
   - ReLU only outputs non-negative values
   - This limits expressiveness - can't represent "opposite" relationships
   - tanh's symmetric range [-1, 1] is more natural for modeling changes

**Why This Matters**:
- **tanh is the standard** for vanilla RNNs and LSTMs for good reasons!
- Some variants use other activations (e.g., GRU uses sigmoid in gates)
- Transformers avoid this issue entirely by not having recurrence

## Part C: Longer Sequences

Let's see how RNNs behave when processing longer sequences (20 timesteps instead of 5).

In [ ]:
# Process a longer sequence
np.random.seed(42)
seq_len_long = 20
inputs_long = np.random.randn(seq_len_long, input_size)

# Use medium hidden size
rnn_long = SimpleRNN(input_size, hidden_size=8, output_size=output_size)
outputs_long, hidden_long = rnn_long.forward(inputs_long)

print(f"Processed sequence of length {seq_len_long}")
print(f"Hidden states shape: {hidden_long.shape}")
print(f"Hidden states range: [{hidden_long.min():.3f}, {hidden_long.max():.3f}]")

In [ ]:
# Visualize longer sequence
fig, axes = plt.subplots(3, 1, figsize=(14, 12))

# Line plot: All dimensions
axes[0].plot(hidden_long, linewidth=2, marker='o', markersize=3)
axes[0].set_xlabel('Timestep', fontsize=12)
axes[0].set_ylabel('Hidden State Value', fontsize=12)
axes[0].set_title('Hidden State Evolution Over 20 Timesteps', fontsize=13, fontweight='bold')
axes[0].legend([f'h[{i}]' for i in range(8)], ncol=4, fontsize=9)
axes[0].grid(True, alpha=0.3)
axes[0].axhline(y=0, color='k', linestyle='--', alpha=0.3)

# Heatmap
sns.heatmap(hidden_long.T, cmap='coolwarm', center=0,
            xticklabels=range(seq_len_long), yticklabels=[f'h[{i}]' for i in range(8)],
            cbar_kws={'label': 'Value'}, ax=axes[1])
axes[1].set_xlabel('Timestep', fontsize=12)
axes[1].set_ylabel('Hidden Dimension', fontsize=12)
axes[1].set_title('Hidden State Heatmap', fontsize=13, fontweight='bold')

# Magnitude over time (L2 norm)
magnitudes = np.linalg.norm(hidden_long, axis=1)
axes[2].plot(magnitudes, linewidth=2.5, marker='o', markersize=5, color='purple')
axes[2].set_xlabel('Timestep', fontsize=12)
axes[2].set_ylabel('Hidden State Magnitude (L2 Norm)', fontsize=12)
axes[2].set_title('Overall Hidden State Magnitude Over Time', fontsize=13, fontweight='bold')
axes[2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

### Analysis: Behavior Over Longer Sequences

**Key Observations**:

1. **Saturation Pattern**:
   - Hidden state values tend to saturate (approach tanh's bounds of ±1)
   - After initial timesteps, changes become smaller
   - This is due to tanh's saturation property

2. **Magnitude Evolution**:
   - The L2 norm shows overall "activity" of the hidden state
   - Often stabilizes after several timesteps
   - This stabilization can be good (stable) or bad (can't adapt to new information)

3. **Information Capacity**:
   - With only 8 dimensions, the RNN must "compress" information from 20 timesteps
   - Early information may be forgotten to make room for new information
   - This is the fundamental limitation that motivates attention mechanisms!

**Why This Matters**:
- Real-world sequences are often 100s or 1000s of timesteps (sentences, documents)
- RNNs struggle to maintain information over such long sequences
- This is why we needed LSTMs (partial solution) and transformers (better solution)

## Exercise 1 Summary

### Key Insights

1. **Hidden Size**:
   - Controls representational capacity
   - Larger = more capacity but more parameters
   - Typical values: 128-1024 for real tasks

2. **Activation Functions**:
   - **tanh is preferred** for RNNs due to bounded range and symmetry
   - ReLU can cause dying neurons and exploding activations
   - Bounded activations (tanh, sigmoid) provide stability

3. **Sequence Length**:
   - RNNs can process arbitrary length sequences
   - But information capacity is limited by hidden_size
   - Longer sequences → more challenging to maintain information

### Connections to Main Concepts

- These experiments show why vanilla RNNs have **limited practical applicability**
- The saturation and information loss motivate **LSTMs** (next exercise)
- The fundamental capacity limitation motivates **attention mechanisms** (future notebook)

**Next**: Let's explore the vanishing gradient problem in depth!

---

# Exercise 2: Understand Vanishing Gradients

## Original Exercise

> **Task**: Play with the gradient simulation.
>
> Questions:
> - What weight value keeps gradients stable longest?
> - Try seq_len=100. Do gradients vanish faster?
> - Why is it hard to find a weight value that avoids both vanishing AND exploding?

## Learning Objectives

- Find the "Goldilocks zone" of weight values
- Understand the exponential nature of gradient decay
- See why RNN training is so sensitive to initialization

Let's investigate!

## Part A: Finding the Stable Weight Value

Let's systematically test different weight values to find which one keeps gradients stable longest.

In [ ]:
# Test a range of weight values
weight_values = [0.5, 0.6, 0.7, 0.8, 0.9, 1.0, 1.1, 1.2]
seq_len = 50

# Store results
gradient_results = {}

for w in weight_values:
    gradients = simulate_gradient_flow(seq_len=seq_len, weight_value=w)
    gradient_results[w] = gradients
    # Print gradient at timestep 0 (earliest) - this shows how much information survived
    print(f"Weight = {w:.1f}: gradient at t=0 = {gradients[0]:.6e}")

print("\n✓ Tested weight values from 0.5 to 1.2")

In [ ]:
# Visualize gradient flow for different weights
fig, axes = plt.subplots(1, 2, figsize=(15, 6))

# LEFT: Logarithmic scale (shows vanishing clearly)
ax = axes[0]
for w in weight_values:
    ax.semilogy(gradient_results[w], label=f'W={w}', linewidth=2.5, marker='o', markersize=3)
ax.set_xlabel('Timestep (0 = earliest, 49 = latest)', fontsize=12)
ax.set_ylabel('Gradient Magnitude (log scale)', fontsize=12)
ax.set_title('Gradient Flow for Different Weight Values\n(Logarithmic Scale)', fontsize=13, fontweight='bold')
ax.legend(fontsize=10, ncol=2)
ax.grid(True, alpha=0.4, which='both')
ax.axhline(y=1e-10, color='r', linestyle='--', alpha=0.5, linewidth=2, label='Essentially zero')

# RIGHT: Bar chart showing gradient at timestep 0
ax = axes[1]
grad_at_t0 = [gradient_results[w][0] for w in weight_values]
colors = ['red' if g < 1e-6 else 'orange' if g < 1e-2 else 'green' if g < 10 else 'red' for g in grad_at_t0]
ax.bar([str(w) for w in weight_values], grad_at_t0, color=colors, alpha=0.7, edgecolor='black')
ax.set_xlabel('Weight Value', fontsize=12)
ax.set_ylabel('Gradient at Timestep 0', fontsize=12)
ax.set_title('Gradient Magnitude at Earliest Timestep\n(Green=Stable, Orange=Vanishing, Red=Extreme)', fontsize=13, fontweight='bold')
ax.set_yscale('log')
ax.grid(True, alpha=0.3, axis='y')
ax.axhline(y=1, color='k', linestyle='--', alpha=0.5, linewidth=2, label='Gradient = 1 (ideal)')
ax.legend()

plt.tight_layout()
plt.show()

### Analysis: The Goldilocks Zone

**Key Findings**:

1. **W = 0.5-0.7** (Red zone - Rapid Vanishing):
   - Gradients vanish extremely fast
   - At timestep 0, gradient ≈ 10^-10 (essentially zero)
   - Cannot learn long-term dependencies at all

2. **W = 0.8-0.9** (Orange zone - Slow Vanishing):
   - Gradients decay more gradually
   - W = 0.9 gives gradient ≈ 10^-3 at timestep 0
   - Best for vanilla RNNs, but still problematic

3. **W = 1.0** (Green zone - Stable!):
   - Gradient stays close to 1.0 throughout
   - This is the "sweet spot" (barely!)
   - But very sensitive - hard to achieve in practice

4. **W > 1.0** (Red zone - Exploding):
   - Gradients explode exponentially
   - W = 1.2 gives gradient ≈ 10^6 at timestep 0
   - Training becomes unstable

**The "Goldilocks Zone"**: W ≈ 0.9-1.0 works best, but it's extremely narrow!

## Part B: Longer Sequences (100 Timesteps)

Do gradients vanish faster with longer sequences? Let's find out!

In [ ]:
# Compare 50 vs 100 timesteps
seq_lengths = [50, 100]
weight_test = 0.9  # Use the "best" weight from Part A

results_by_length = {}

for seq_len in seq_lengths:
    gradients = simulate_gradient_flow(seq_len=seq_len, weight_value=weight_test)
    results_by_length[seq_len] = gradients
    print(f"Sequence length {seq_len:3d}: gradient at t=0 = {gradients[0]:.6e}")

# Calculate the ratio
ratio = results_by_length[100][0] / results_by_length[50][0]
print(f"\nGradient ratio (100-step / 50-step): {ratio:.6e}")
print(f"This means the gradient is {1/ratio:.1f}x smaller for longer sequences!")

In [ ]:
# Visualize the comparison
fig, axes = plt.subplots(1, 2, figsize=(15, 6))

# LEFT: Overlaid plots (log scale)
ax = axes[0]
for seq_len in seq_lengths:
    ax.semilogy(results_by_length[seq_len], 
                label=f'{seq_len} timesteps',
                linewidth=3, marker='o', markersize=4)
ax.set_xlabel('Timestep', fontsize=12)
ax.set_ylabel('Gradient Magnitude (log scale)', fontsize=12)
ax.set_title(f'Gradient Decay: 50 vs 100 Timesteps\n(Weight = {weight_test})', 
             fontsize=13, fontweight='bold')
ax.legend(fontsize=12)
ax.grid(True, alpha=0.4, which='both')
ax.axhline(y=1e-10, color='r', linestyle='--', alpha=0.5, label='Essentially zero')

# RIGHT: Direct comparison of first 50 timesteps
ax = axes[1]
ax.plot(range(50), results_by_length[50][:50], 
        label='50-step sequence', linewidth=3, marker='o', markersize=4)
ax.plot(range(50), results_by_length[100][:50], 
        label='100-step sequence (first 50)', linewidth=3, marker='s', markersize=4)
ax.set_xlabel('Timestep', fontsize=12)
ax.set_ylabel('Gradient Magnitude (linear scale)', fontsize=12)
ax.set_title('First 50 Timesteps: Same Decay Rate\n(But 100-step starts from lower base)', 
             fontsize=13, fontweight='bold')
ax.legend(fontsize=12)
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

### Analysis: Exponential Relationship

**Key Finding**: Yes, gradients vanish **much faster** with longer sequences!

**Why**:
- Each additional timestep multiplies the gradient by approximately (W × tanh') ≈ 0.45 (for W=0.9)
- After 50 steps: gradient ≈ (0.45)^50 ≈ 10^-3
- After 100 steps: gradient ≈ (0.45)^100 ≈ (10^-3)^2 ≈ 10^-6
- The relationship is **exponential**, not linear!

**Practical Implication**:
```
Sequence length   Gradient at t=0   Can learn?
     10 steps        ~0.1            ✓ Yes
     20 steps        ~0.01           ✓ Maybe
     50 steps        ~10^-3          ✗ Difficult
    100 steps        ~10^-6          ✗ No
    500 steps        ~10^-30         ✗ Impossible
```

**This is why vanilla RNNs are limited to ~10-20 timesteps in practice!**

## Part C: The Dilemma - Why It's So Hard

Why can't we just pick the "right" weight value to avoid both vanishing and exploding? Let's visualize the problem.

In [ ]:
# Create a detailed analysis of the stability region
weight_values_fine = np.linspace(0.4, 1.3, 50)  # Finer grid
seq_len = 50

# For each weight, compute gradient at t=0
final_gradients = []

for w in weight_values_fine:
    grads = simulate_gradient_flow(seq_len=seq_len, weight_value=w)
    final_gradients.append(grads[0])

final_gradients = np.array(final_gradients)

In [ ]:
# Visualize the stability region
fig, axes = plt.subplots(2, 1, figsize=(14, 10))

# TOP: Gradient vs Weight (log scale)
ax = axes[0]
ax.semilogy(weight_values_fine, final_gradients, linewidth=3, color='darkblue')
ax.fill_between(weight_values_fine, 1e-10, final_gradients, 
                 where=(final_gradients < 1e-4), alpha=0.3, color='red', label='Vanishing')
ax.fill_between(weight_values_fine, final_gradients, 1e10,
                 where=(final_gradients > 10), alpha=0.3, color='orange', label='Exploding')
ax.fill_between(weight_values_fine, 1e-10, 1e10,
                 where=((final_gradients >= 1e-4) & (final_gradients <= 10)), 
                 alpha=0.3, color='green', label='Stable')
ax.axhline(y=1, color='k', linestyle='--', linewidth=2, alpha=0.7, label='Ideal (gradient=1)')
ax.set_xlabel('Weight Value (W)', fontsize=13)
ax.set_ylabel('Gradient at Timestep 0 (log scale)', fontsize=13)
ax.set_title('The Vanishing/Exploding Gradient Dilemma\n(50 timesteps, tanh activation)', 
             fontsize=14, fontweight='bold')
ax.legend(fontsize=11, loc='upper left')
ax.grid(True, alpha=0.4, which='both')
ax.set_xlim([0.4, 1.3])

# BOTTOM: Zoomed in on the stable region
ax = axes[1]
# Find stable region
stable_mask = (final_gradients >= 1e-4) & (final_gradients <= 10)
stable_weights = weight_values_fine[stable_mask]
stable_grads = final_gradients[stable_mask]

if len(stable_weights) > 0:
    ax.plot(weight_values_fine, final_gradients, linewidth=3, color='darkblue')
    ax.axhline(y=1, color='k', linestyle='--', linewidth=2, alpha=0.7)
    ax.axhline(y=1e-4, color='r', linestyle='--', linewidth=1.5, alpha=0.5, label='Vanishing threshold')
    ax.axhline(y=10, color='orange', linestyle='--', linewidth=1.5, alpha=0.5, label='Exploding threshold')
    ax.fill_between(stable_weights, 0, 20, alpha=0.2, color='green', label='Stable zone')
    ax.set_ylim([1e-5, 100])
    ax.set_yscale('log')
    ax.set_xlabel('Weight Value (W)', fontsize=13)
    ax.set_ylabel('Gradient (log scale)', fontsize=13)
    ax.set_title(f'The Narrow "Goldilocks Zone": W ≈ {stable_weights.min():.2f} to {stable_weights.max():.2f}',
                 fontsize=14, fontweight='bold')
    ax.legend(fontsize=11)
    ax.grid(True, alpha=0.4, which='both')
    ax.set_xlim([0.4, 1.3])
    
    # Annotate the stable range
    stable_range = stable_weights.max() - stable_weights.min()
    ax.text(0.95, 1.5, f'Stable range width:\n~{stable_range:.3f}',
            transform=ax.transData, fontsize=12,
            bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.8),
            ha='center')

plt.tight_layout()
plt.show()

### Analysis: Why the Dilemma Exists

**The Problem**: The "stable zone" (green) is **extremely narrow**!

**Mathematical Explanation**:

1. **Gradient flow equation**:
   ```
   gradient(t=0) = gradient(t=T) × ∏(t=0 to T-1) [W × tanh'(...)]
   ```

2. **For vanilla RNN**:
   - Each timestep multiplies by: `W × tanh'` ≈ `W × 0.5`
   - After T steps: gradient ≈ `(W × 0.5)^T`

3. **For gradient to stay near 1**:
   - Need: `(W × 0.5)^T ≈ 1`
   - Solving: `W × 0.5 ≈ 1` → `W ≈ 2.0`
   - But wait... this assumes tanh' is constant!

4. **The reality**:
   - `tanh'(x)` varies from 0 to 1 depending on x
   - Average is ~0.5, but actual values fluctuate
   - So we need W ≈ 0.9-1.0 to compensate

5. **Why it's hard**:
   ```
   If W = 0.9:  (0.9 × 0.5)^50 = 0.45^50 ≈ 10^-3   → Vanishing
   If W = 1.0:  (1.0 × 0.5)^50 = 0.5^50  ≈ 10^-15  → Still vanishing!
   If W = 1.1:  (1.1 × 0.5)^50 = 0.55^50 ≈ 10^-11  → Still vanishing!
   If W = 1.5:  (1.5 × 0.5)^50 = 0.75^50 ≈ 10^-6   → Exploding!
   ```

**Why We Can't Just "Initialize Better"**:
- The stable range is ~0.05 wide (e.g., 0.92 to 0.97)
- Random initialization can't guarantee landing in this range
- During training, weights will drift out of the stable zone
- Different layers/dimensions need different optimal weights

**This is THE fundamental problem that LSTMs were designed to solve!**

## Exercise 2 Summary

### Key Insights

1. **Optimal Weight Value**: W ≈ 0.9-1.0
   - This range keeps gradients most stable
   - But even this is not sufficient for long sequences
   - The stable zone is extremely narrow (~0.05 wide)

2. **Sequence Length Impact**: Exponential decay
   - Doubling sequence length doesn't double the problem - it squares it!
   - 100 timesteps: gradient ≈ (gradient at 50 steps)²
   - Practically limits vanilla RNNs to ~10-20 timesteps

3. **The Vanishing/Exploding Dilemma**:
   - Too small W → gradients vanish
   - Too large W → gradients explode
   - "Just right" W is nearly impossible to maintain
   - This is why careful initialization (e.g., Xavier, He) helps but doesn't solve the problem

### Why This Matters

**The vanishing gradient problem is not a bug - it's a fundamental mathematical property of deep recurrent architectures!**

Solutions attempted:
1. ✗ **Better initialization**: Helps but doesn't solve it
2. ✗ **Gradient clipping**: Prevents exploding but not vanishing
3. ✓ **LSTMs/GRUs**: Partially solve with gates and cell state (next exercise!)
4. ✓ **Transformers**: Completely avoid the problem with attention

**Next**: Let's see how LSTM gates provide more fine-grained control!

---

# Exercise 3: Interpret LSTM Gates

## Original Exercise

> **Task**: Look at the gate visualizations and answer:
>
> - Which gate values (0 or 1) indicate "remembering" information?
> - Which indicate "forgetting"?
> - How might gate patterns differ for:
>   - A sequence that requires long-term memory?
>   - A sequence where you want to quickly forget old information?

## Learning Objectives

- Understand what each LSTM gate controls
- Interpret gate activation patterns
- See how gates adapt to different sequence types

Let's explore LSTM's intelligent memory control!

## Part A: What Do Gate Values Mean?

### Quick Refresher: The Three Gates

1. **Forget Gate (f_t)**: Controls what to forget from cell state
   - f_t ≈ 1 → **Remember** (keep old information)
   - f_t ≈ 0 → **Forget** (discard old information)

2. **Input Gate (i_t)**: Controls what new info to add
   - i_t ≈ 1 → **Accept** new information
   - i_t ≈ 0 → **Ignore** new information

3. **Output Gate (o_t)**: Controls what to output
   - o_t ≈ 1 → **Output** this information
   - o_t ≈ 0 → **Suppress** this information

Let's see these in action!

In [ ]:
# Create LSTM and process a simple sequence
np.random.seed(42)
lstm = LSTMCell(input_size=3, hidden_size=4)

# Process a sequence and track everything
seq_len = 10
inputs = np.random.randn(seq_len, 3)

# Initialize
h = np.zeros(4)
C = np.zeros(4)

# Storage
forget_gates = []
input_gates = []
output_gates = []
cell_states = []
hidden_states = []

for t in range(seq_len):
    h, C, (f_t, i_t, o_t) = lstm.forward(inputs[t], h, C)
    
    forget_gates.append(f_t.copy())
    input_gates.append(i_t.copy())
    output_gates.append(o_t.copy())
    cell_states.append(C.copy())
    hidden_states.append(h.copy())

# Convert to arrays
forget_gates = np.array(forget_gates)
input_gates = np.array(input_gates)
output_gates = np.array(output_gates)
cell_states = np.array(cell_states)

print(f"Processed {seq_len} timesteps through LSTM")
print(f"Forget gates shape: {forget_gates.shape}")
print(f"\nAverage gate values:")
print(f"  Forget gate: {forget_gates.mean():.3f} (1=remember, 0=forget)")
print(f"  Input gate:  {input_gates.mean():.3f} (1=accept new, 0=ignore new)")
print(f"  Output gate: {output_gates.mean():.3f} (1=output, 0=suppress)")

In [ ]:
# Visualize gate patterns
fig, axes = plt.subplots(2, 2, figsize=(15, 11))

# Forget Gate
sns.heatmap(forget_gates.T, ax=axes[0, 0], cmap='RdYlGn', vmin=0, vmax=1,
            xticklabels=range(seq_len), yticklabels=range(4),
            cbar_kws={'label': 'Gate Value'}, annot=True, fmt='.2f')
axes[0, 0].set_xlabel('Timestep')
axes[0, 0].set_ylabel('Hidden Dimension')
axes[0, 0].set_title('Forget Gate f_t\n(Green=Remember, Red=Forget)', fontweight='bold')

# Input Gate
sns.heatmap(input_gates.T, ax=axes[0, 1], cmap='RdYlGn', vmin=0, vmax=1,
            xticklabels=range(seq_len), yticklabels=range(4),
            cbar_kws={'label': 'Gate Value'}, annot=True, fmt='.2f')
axes[0, 1].set_xlabel('Timestep')
axes[0, 1].set_ylabel('Hidden Dimension')
axes[0, 1].set_title('Input Gate i_t\n(Green=Accept New, Red=Ignore)', fontweight='bold')

# Output Gate
sns.heatmap(output_gates.T, ax=axes[1, 0], cmap='RdYlGn', vmin=0, vmax=1,
            xticklabels=range(seq_len), yticklabels=range(4),
            cbar_kws={'label': 'Gate Value'}, annot=True, fmt='.2f')
axes[1, 0].set_xlabel('Timestep')
axes[1, 0].set_ylabel('Hidden Dimension')
axes[1, 0].set_title('Output Gate o_t\n(Green=Output, Red=Suppress)', fontweight='bold')

# Cell State
sns.heatmap(cell_states.T, ax=axes[1, 1], cmap='coolwarm', center=0,
            xticklabels=range(seq_len), yticklabels=range(4),
            cbar_kws={'label': 'Cell Value'}, annot=True, fmt='.2f')
axes[1, 1].set_xlabel('Timestep')
axes[1, 1].set_ylabel('Hidden Dimension')
axes[1, 1].set_title('Cell State C_t (Long-term Memory)\n(Red=Positive, Blue=Negative)', fontweight='bold')

plt.suptitle('LSTM Gate Activations Over Time', fontsize=14, fontweight='bold', y=1.00)
plt.tight_layout()
plt.show()

### Interpreting the Patterns

**What We See** (for random input):

1. **Forget Gate** (Top-left):
   - Values around 0.5 (yellow-green)
   - This means: "partially remember, partially forget"
   - The LSTM is being cautious - not committing fully

2. **Input Gate** (Top-right):
   - Also around 0.5
   - This means: "partially accept new information"
   - Balanced approach to incorporating new inputs

3. **Output Gate** (Bottom-left):
   - Similar pattern
   - Controls what gets output to next layer

4. **Cell State** (Bottom-right):
   - This is the actual long-term memory!
   - Values can be positive or negative
   - Notice how it evolves based on forget and input gates

**Key Insight**: For random input, gates hover around 0.5. But for **meaningful sequences**, gates should show clear patterns! Let's test this...

## Part B: Gate Patterns for Different Sequence Types

Let's create two synthetic sequences with very different requirements and see how a trained LSTM would behave.

### Scenario 1: Long-Term Dependency Task
**Task**: Remember the first input for all 10 timesteps.

In [ ]:
# Scenario 1: Long-term dependency
# Sequence: [1, 0, 0, 0, 0, 0, 0, 0, 0, 0]
# Goal: Remember that first "1" throughout

np.random.seed(42)
lstm_longterm = LSTMCell(input_size=1, hidden_size=4)

# Create sequence: important signal at t=0, then zeros
seq_longterm = np.zeros((10, 1))
seq_longterm[0, 0] = 1.0  # Important information at start!

# Process and track
h = np.zeros(4)
C = np.zeros(4)

f_gates_lt = []
i_gates_lt = []
o_gates_lt = []
C_states_lt = []

for t in range(len(seq_longterm)):
    h, C, (f_t, i_t, o_t) = lstm_longterm.forward(seq_longterm[t], h, C)
    f_gates_lt.append(f_t.copy())
    i_gates_lt.append(i_t.copy())
    o_gates_lt.append(o_t.copy())
    C_states_lt.append(C.copy())

f_gates_lt = np.array(f_gates_lt)
i_gates_lt = np.array(i_gates_lt)
o_gates_lt = np.array(o_gates_lt)
C_states_lt = np.array(C_states_lt)

print("Scenario 1: Long-term dependency")
print(f"  Average forget gate after t=0: {f_gates_lt[1:].mean():.3f}")
print(f"  Average input gate after t=0: {i_gates_lt[1:].mean():.3f}")
print("  → High forget gate would mean 'keep remembering the initial 1'")
print("  → Low input gate would mean 'ignore the zeros that follow'")

### Scenario 2: Rapidly Changing Task
**Task**: Track a counting sequence where each new number matters.

In [ ]:
# Scenario 2: Rapidly changing
# Sequence: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
# Goal: Keep updating with new information

np.random.seed(42)
lstm_rapid = LSTMCell(input_size=1, hidden_size=4)

# Create sequence: constantly changing
seq_rapid = np.arange(1, 11).reshape(-1, 1).astype(float)

# Process and track
h = np.zeros(4)
C = np.zeros(4)

f_gates_rp = []
i_gates_rp = []
o_gates_rp = []
C_states_rp = []

for t in range(len(seq_rapid)):
    h, C, (f_t, i_t, o_t) = lstm_rapid.forward(seq_rapid[t], h, C)
    f_gates_rp.append(f_t.copy())
    i_gates_rp.append(i_t.copy())
    o_gates_rp.append(o_t.copy())
    C_states_rp.append(C.copy())

f_gates_rp = np.array(f_gates_rp)
i_gates_rp = np.array(i_gates_rp)
o_gates_rp = np.array(o_gates_rp)
C_states_rp = np.array(C_states_rp)

print("Scenario 2: Rapidly changing")
print(f"  Average forget gate: {f_gates_rp.mean():.3f}")
print(f"  Average input gate: {i_gates_rp.mean():.3f}")
print("  → Ideally: low forget gate (don't hold onto old) + high input gate (accept new)")

In [ ]:
# Compare gate patterns side-by-side
fig, axes = plt.subplots(2, 3, figsize=(16, 10))

scenarios = [
    ('Long-term Dependency\n(Remember first input)', f_gates_lt, i_gates_lt, o_gates_lt),
    ('Rapidly Changing\n(Update constantly)', f_gates_rp, i_gates_rp, o_gates_rp)
]

for row, (title, f_g, i_g, o_g) in enumerate(scenarios):
    # Forget gate
    sns.heatmap(f_g.T, ax=axes[row, 0], cmap='RdYlGn', vmin=0, vmax=1,
                xticklabels=range(10), yticklabels=range(4),
                cbar_kws={'label': 'Value'}, annot=True, fmt='.2f')
    axes[row, 0].set_xlabel('Timestep')
    axes[row, 0].set_ylabel('Dimension')
    axes[row, 0].set_title(f'Forget Gate\n{title}', fontsize=11, fontweight='bold')
    
    # Input gate
    sns.heatmap(i_g.T, ax=axes[row, 1], cmap='RdYlGn', vmin=0, vmax=1,
                xticklabels=range(10), yticklabels=range(4),
                cbar_kws={'label': 'Value'}, annot=True, fmt='.2f')
    axes[row, 1].set_xlabel('Timestep')
    axes[row, 1].set_ylabel('Dimension')
    axes[row, 1].set_title(f'Input Gate\n{title}', fontsize=11, fontweight='bold')
    
    # Output gate
    sns.heatmap(o_g.T, ax=axes[row, 2], cmap='RdYlGn', vmin=0, vmax=1,
                xticklabels=range(10), yticklabels=range(4),
                cbar_kws={'label': 'Value'}, annot=True, fmt='.2f')
    axes[row, 2].set_xlabel('Timestep')
    axes[row, 2].set_ylabel('Dimension')
    axes[row, 2].set_title(f'Output Gate\n{title}', fontsize=11, fontweight='bold')

plt.suptitle('LSTM Gate Patterns for Different Sequence Types', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

### Analysis: What Should We Expect?

**Note**: The LSTM above is **untrained** (random weights), so we don't see ideal patterns yet. But let's discuss what a **trained** LSTM would learn:

#### For Long-term Dependency Task:
**Ideal behavior**:
1. **t=0** (seeing the important "1"):
   - High input gate (i_t ≈ 1): "Accept this important information!"
   - Forget gate can be low: "Nothing to remember from before"

2. **t=1 to 9** (seeing zeros):
   - **High forget gate** (f_t ≈ 1): "Keep remembering what I stored at t=0!"
   - **Low input gate** (i_t ≈ 0): "Ignore these meaningless zeros"
   - Output gate: depends on task

**Why this works**: Cell state equation is `C_t = f_t * C_{t-1} + i_t * C_tilde`
- If f_t ≈ 1 and i_t ≈ 0: `C_t ≈ C_{t-1}` → **Memory is preserved!**

#### For Rapidly Changing Task:
**Ideal behavior**:
1. **Every timestep**:
   - **Moderate forget gate** (f_t ≈ 0.3-0.7): "Partially forget old count"
   - **High input gate** (i_t ≈ 0.7-1.0): "Strongly accept new count"
   - This allows the model to update quickly

**Why this works**:
- If f_t ≈ 0.5 and i_t ≈ 0.8: `C_t ≈ 0.5 * C_{t-1} + 0.8 * new_info`
- Old information fades, new information dominates

### Key Insight

**LSTMs learn to modulate their gates based on the task!**
- Long-term tasks → high forget gates (remember)
- Short-term tasks → low forget gates (quickly update)
- The gates provide **fine-grained control** over information flow

## Exercise 3 Summary

### Key Insights

1. **Gate Value Meanings**:
   - **Forget gate**: 1=remember, 0=forget
   - **Input gate**: 1=accept new info, 0=ignore new info
   - **Output gate**: 1=output this, 0=suppress

2. **Task-Dependent Patterns**:
   - **Long-term memory tasks**: High forget gates + low input gates
     - Preserves information over many timesteps
     - `C_t ≈ C_0` (cell state stays constant)
   
   - **Rapidly changing tasks**: Low forget gates + high input gates
     - Quickly updates to new information
     - `C_t ≈ new_info` (cell state tracks current input)

3. **Fine-Grained Control**:
   - Unlike vanilla RNN with single hidden state
   - LSTM has 3 independent gates per dimension
   - Can simultaneously: remember some dims, forget others, update others
   - This is why LSTMs handle ~50-100 timesteps vs RNN's ~10-20

### Why This Matters

**LSTMs provide adaptive memory management**:
- Vanilla RNN: Fixed update rule `h_t = tanh(W @ h_{t-1} + ...)`
- LSTM: Learned gates decide what to remember/forget/update
- This flexibility enables longer-term dependencies

**But LSTMs still have limitations**:
- Gates help but don't fully solve vanishing gradients
- Still sequential (can't parallelize)
- Still have information bottleneck (fixed hidden size)

This motivates **attention mechanisms** and **transformers**!

**Next**: Let's think conceptually about how to solve these remaining problems.

---

# Exercise 4: Think Ahead to Transformers

## Original Exercise

> **Task**: Before seeing the next notebook, brainstorm:
>
> - How could we give a model "direct access" to all inputs at once?
> - How might we process an entire sequence in parallel?
> - Sketch your ideas on paper!

## Learning Objectives

- Think creatively about the problems we've identified
- Understand the conceptual gaps that transformers fill
- Prepare mentally for the attention mechanism

This is a **conceptual exercise** - there's no single right answer! The goal is to think about the problems from first principles.

## Review: What Problems Do We Need to Solve?

From our previous exercises, we've identified these fundamental limitations of RNNs/LSTMs:

### Problem 1: Sequential Processing
```
RNN/LSTM:
  t=0 → t=1 → t=2 → t=3 → t=4 → t=5
  (must wait for each step before proceeding)

❌ Slow: Can't use modern GPUs effectively
❌ Serial: 100-word sentence = 100 sequential steps
```

### Problem 2: No Direct Access
```
To connect word at position 1 to word at position 50:
  Information must flow through 49 intermediate steps
  Each step degrades the signal (even with LSTMs)

❌ Indirect path: Information gets corrupted
❌ Long-range: Harder as distance increases
```

### Problem 3: Fixed-Size Bottleneck
```
In seq2seq:
  100-word input → compress to fixed vector → generate output
  
❌ Information loss: Can't fit everything in fixed size
❌ Length-dependent: Quality degrades with longer inputs
```

### Problem 4: Limited Attention
```
When processing word 50, RNN has:
  - Strong memory of words 45-50 (recent)
  - Weak memory of words 1-10 (distant)
  - No way to "look back" directly

❌ Recency bias: Recent words dominate
❌ No selective focus: Can't choose what to focus on
```

Let's brainstorm solutions!

## Brainstorm 1: How to Give "Direct Access" to All Inputs?

### Idea 1: Random Access Memory
```
Concept: Store all inputs in a memory bank with index-based lookup

Structure:
  Memory = [word_1, word_2, word_3, ..., word_N]
  
At any timestep t, can access any word_i directly:
  - No need to propagate through intermediate steps
  - Like RAM in a computer: O(1) access time

Example:
  Processing word_50? 
  Can directly look at word_1 without going through word_2...word_49!

Pros:
  ✓ Direct access - no information degradation
  ✓ No distance problem
  
Cons:
  ✗ How to decide WHICH words to access?
  ✗ How to combine information from multiple words?
  ✗ Still sequential if we process one word at a time
```

### Idea 2: Fully Connected Graph
```
Concept: Connect every word to every other word

Structure:
  word_1 ←→ word_2
  word_1 ←→ word_3
  word_1 ←→ word_4
  ...
  word_2 ←→ word_3
  word_2 ←→ word_4
  ...
  (Every word connected to every other word!)

Example:
  word_1 can directly influence word_50
  word_50 can directly influence word_1
  
Pros:
  ✓ Every word can "see" every other word
  ✓ No distance limitation
  ✓ Information flows in all directions

Cons:
  ✗ Massive number of connections: N^2 for N words
  ✗ How to weight/combine all these connections?
  ✗ Still need some way to aggregate information
```

### Idea 3: Attention-Based Lookup
```
Concept: When processing word_i, compute a "relevance score" for all other words

Intuition:
  "Which other words should I pay attention to?"
  
Example - Translating "The cat sat on the mat":
  Processing "cat":
    - High relevance to "The" (subject marker)
    - High relevance to "sat" (verb, what did cat do?)
    - Low relevance to "on" (preposition, less important)
    - Medium relevance to "mat" (location)

Mechanism:
  For each word:
    1. Compute relevance scores to ALL other words
    2. Weight other words by relevance
    3. Combine weighted information

Pros:
  ✓ Direct access to all words
  ✓ Learned relevance (not fixed)
  ✓ Can focus on important words, ignore irrelevant ones

Cons:
  ✗ How to compute "relevance"?
  ✗ Still seems complex...

💡 This is basically the ATTENTION MECHANISM!
```

### Your Ideas?

Before moving forward, think:
- Which of these ideas seems most promising?
- Can you think of other approaches?
- What are the key challenges to solve?

## Brainstorm 2: How to Process Sequences in Parallel?

### Current Problem: Sequential Dependency
```
RNN/LSTM:
  h_1 = f(x_1, h_0)      ← depends on h_0
  h_2 = f(x_2, h_1)      ← depends on h_1 ← can't compute until h_1 done
  h_3 = f(x_3, h_2)      ← depends on h_2 ← can't compute until h_2 done
  ...

This is INHERENTLY SEQUENTIAL.
Can't compute h_3 until h_2 is done.
```

### Idea 1: Remove Recurrence Entirely
```
Concept: Don't use previous hidden states at all!

Instead:
  h_1 = f(x_1, ALL_OTHER_WORDS)
  h_2 = f(x_2, ALL_OTHER_WORDS)
  h_3 = f(x_3, ALL_OTHER_WORDS)
  ...

Key insight:
  Each h_i can be computed INDEPENDENTLY!
  No dependency on h_{i-1}
  
Parallelization:
  Compute all h_i simultaneously using matrix operations
  
Example (for 4 words):
  h_1, h_2, h_3, h_4 = parallel_function([x_1, x_2, x_3, x_4])
  
Pros:
  ✓ Fully parallel - no waiting
  ✓ Can leverage GPUs (great at parallel matrix ops)
  ✓ Faster training and inference

Cons:
  ✗ How does each word know about context?
  ✗ Isn't the "memory" important?
  → Solution: Let each word look at ALL other words! (attention again!)
```

### Idea 2: Matrix Operations Instead of Loops
```
Concept: Replace for-loop over timesteps with matrix multiplication

Current (sequential):
  for t in range(seq_len):
      h_t = process(x_t, h_{t-1})

New (parallel):
  H = matrix_operation(X)  # Process entire sequence at once!
  
Where:
  X = [x_1, x_2, ..., x_N]  (entire input sequence)
  H = [h_1, h_2, ..., h_N]  (entire output sequence)
  
Matrix operations are highly parallelizable on GPUs!

Example:
  Instead of:
    for i in range(1000):  # 1000 sequential steps
        result[i] = compute(input[i])
  
  Do:
    result = matrix_multiply(weights, input)  # Single parallel operation!
```

### Idea 3: Position-Aware Processing
```
Problem: If we remove recurrence, how does the model know word order?
  "cat dog bit" vs "dog cat bit" should be different!
  But if processing in parallel, how to encode position?

Solution: Add position information to each word!

Concept:
  word_embedding = content_embedding + position_embedding
  
Example:
  "cat" at position 1: [cat_vector] + [position_1_vector]
  "cat" at position 5: [cat_vector] + [position_5_vector]
  
Now each word knows:
  - What it is (content)
  - Where it is (position)
  
This allows parallel processing while preserving order!

💡 This is POSITIONAL ENCODING in transformers!
```

### The Big Insight

**Transformers combine these ideas**:
1. Remove recurrence → enable parallelization
2. Add attention → give direct access to all words
3. Add positional encodings → preserve order information

Result:
- ✓ Parallel processing (fast!)
- ✓ Direct connections (no degradation!)
- ✓ Position-aware (understands order!)
- ✓ Selective focus (attention to relevant words!)

## Visualization: RNN vs Transformer Architecture

### RNN/LSTM Architecture
```
Sequential Processing:

Time:    t=0      t=1      t=2      t=3
         ↓        ↓        ↓        ↓
Input:  [The]  → [cat]  → [sat]  → [down]
         ↓        ↓        ↓        ↓  
RNN:    h_0 --→ h_1 --→ h_2 --→ h_3
         ↓        ↓        ↓        ↓
Output: y_0     y_1      y_2      y_3

Information flow:
- "The" must pass through h_0 → h_1 → h_2 → h_3 to reach "down"
- Sequential: Can't start h_2 until h_1 finishes
- Distance problem: Information from "The" degrades by time it reaches "down"
```

### Transformer Architecture (Preview)
```
Parallel Processing:

Input:  [The]    [cat]    [sat]    [down]
         ↓        ↓        ↓        ↓
      Encode   Encode   Encode   Encode  (Add position info)
         ↓        ↓        ↓        ↓
         └────────┴────────┴────────┘
                  ↓
         Self-Attention Layer
         (Every word looks at every word)
                  ↓
         ┌────────┬────────┬────────┐
         ↓        ↓        ↓        ↓
       out_0    out_1    out_2    out_3

Information flow:
- "The" can DIRECTLY access "down" (and vice versa)
- Parallel: All words processed simultaneously
- No distance problem: Direct connections everywhere
- Attention: Each word decides which others are relevant
```

### Key Differences

| Aspect | RNN/LSTM | Transformer |
|--------|----------|-------------|
| **Processing** | Sequential | Parallel |
| **Access** | Indirect (through chain) | Direct (all-to-all) |
| **Speed** | Slow (must wait) | Fast (parallel) |
| **Long-range** | Difficult (degrades) | Easy (direct) |
| **Bottleneck** | Fixed hidden size | Attention to all words |
| **Position** | Implicit (order of processing) | Explicit (positional encoding) |

## Exercise 4 Summary

### Key Conceptual Insights

1. **Direct Access Problem**:
   - RNNs force indirect access through sequential chain
   - Solution ideas: memory bank, fully connected graph, **attention mechanism**
   - Attention lets each word compute relevance to all others

2. **Parallel Processing Problem**:
   - Recurrence creates sequential dependency
   - Solution: Remove recurrence, use matrix operations
   - Add positional encodings to preserve order

3. **The Transformer Approach**:
   - No recurrence → parallelizable
   - Attention → direct access
   - Positional encoding → order-aware
   - Result: Fast, powerful, scalable

### Why This Mental Model Matters

When you learn about transformers in the next notebook:
- **Self-attention** will seem natural (we brainstormed this!)
- **Positional encoding** will make sense (needed for parallel processing)
- **Multi-head attention** will be a logical extension

You now understand **WHY** transformers were invented:
1. RNNs are slow (sequential)
2. RNNs struggle with long sequences (vanishing gradients)
3. RNNs have information bottlenecks (fixed hidden size)
4. We need: parallel + direct access + selective focus

### Preparing for the Next Notebook

In `02_attention_mechanism.ipynb`, you'll learn:
- How attention computes "relevance scores"
- What Query, Key, and Value mean
- How self-attention enables all-to-all connections
- How to implement attention from scratch
- Why "Attention Is All You Need"

You're now mentally prepared for one of the most important ideas in modern AI! 🚀

**Next**: Before we move to transformers, let's build something practical - a character-level language model with full backpropagation!

---

# Exercise 5: Build a Character-Level Language Model

## Original Exercise

> **Task (Advanced)**: Implement a character-level language model.
>
> - Train an RNN to predict the next character
> - Given: "hell" → predict: "o"
> - Given: "hello worl" → predict: "d"
>
> **Hints**:
> - Use the `SimpleRNN` class we built
> - Start with a very short text ("hello world")
> - One-hot encode characters
> - See how well it learns!

## What We'll Build

We'll create a **complete working language model** with:
1. **Data preparation**: Vocabulary, one-hot encoding, training pairs
2. **Architecture**: CharRNN with output layer for prediction
3. **Backpropagation**: Full BPTT (Backpropagation Through Time) implementation
4. **Training**: Gradient descent with loss tracking
5. **Evaluation**: Text generation and analysis

This is challenging but incredibly rewarding - you'll implement the core algorithm behind modern LLMs!

Let's build it step by step. 🚀

## Step 1: Data Preparation

First, let's prepare our training data.

In [ ]:
# Training text: Start simple!
training_text = "hello world"

print(f"Training text: '{training_text}'")
print(f"Length: {len(training_text)} characters")

# Build vocabulary
chars = sorted(list(set(training_text)))
vocab_size = len(chars)

print(f"\nVocabulary ({vocab_size} unique characters):")
print(chars)

# Create character mappings
char_to_idx = {ch: i for i, ch in enumerate(chars)}
idx_to_char = {i: ch for i, ch in enumerate(chars)}

print(f"\nCharacter → Index mapping:")
for ch, idx in char_to_idx.items():
    print(f"  '{ch}' → {idx}")

### One-Hot Encoding

We need to convert characters to numerical vectors that the RNN can process.

In [ ]:
def char_to_onehot(char, char_to_idx, vocab_size):
    """
    Convert a character to a one-hot encoded vector.
    
    Args:
        char: Character to encode
        char_to_idx: Dictionary mapping characters to indices
        vocab_size: Size of vocabulary
    
    Returns:
        One-hot vector of shape (vocab_size,)
    """
    vec = np.zeros(vocab_size)
    vec[char_to_idx[char]] = 1.0
    return vec

def onehot_to_char(onehot, idx_to_char):
    """
    Convert a one-hot vector (or probability distribution) to a character.
    
    Args:
        onehot: One-hot vector or probability distribution
        idx_to_char: Dictionary mapping indices to characters
    
    Returns:
        Character corresponding to the highest probability
    """
    idx = np.argmax(onehot)
    return idx_to_char[idx]

# Test encoding
test_char = 'h'
test_vec = char_to_onehot(test_char, char_to_idx, vocab_size)

print(f"One-hot encoding of '{test_char}':")
print(test_vec)
print(f"\nDecoding back: '{onehot_to_char(test_vec, idx_to_char)}'")
print("\n✓ One-hot encoding works!")

### Create Training Pairs

We'll create sliding windows: given a sequence of characters, predict the next one.

In [ ]:
def create_training_data(text, char_to_idx, vocab_size, seq_length=3):
    """
    Create input-target pairs for training.
    
    Args:
        text: Training text
        char_to_idx: Character to index mapping
        vocab_size: Size of vocabulary
        seq_length: Length of input sequences
    
    Returns:
        inputs: List of input sequences (each seq_length characters)
        targets: List of target characters (next character after sequence)
    """
    inputs = []
    targets = []
    
    # Sliding window over text
    for i in range(len(text) - seq_length):
        # Input: seq_length characters
        input_seq = text[i:i+seq_length]
        # Target: next character
        target_char = text[i+seq_length]
        
        # Convert to one-hot
        input_onehot = np.array([char_to_onehot(ch, char_to_idx, vocab_size) 
                                  for ch in input_seq])
        target_idx = char_to_idx[target_char]
        
        inputs.append(input_onehot)
        targets.append(target_idx)
    
    return inputs, targets

# Create training data with sequence length 3
seq_length = 3
inputs, targets = create_training_data(training_text, char_to_idx, vocab_size, seq_length)

print(f"Created {len(inputs)} training pairs with sequence length {seq_length}\n")
print("Examples:")
for i in range(min(5, len(inputs))):
    # Decode input sequence
    input_chars = ''.join([onehot_to_char(inputs[i][j], idx_to_char) 
                           for j in range(seq_length)])
    target_char = idx_to_char[targets[i]]
    print(f"  Input: '{input_chars}' → Target: '{target_char}'")

print(f"\n✓ Training data ready: {len(inputs)} examples")

## Step 2: CharRNN Architecture

Now let's build our character-level RNN with an output layer for predictions.

In [ ]:
class CharRNN:
    """
    Character-level RNN for language modeling.
    
    Architecture:
      Input (one-hot) → RNN → Output layer → Predictions
    
    Implements full backpropagation through time (BPTT).
    """
    
    def __init__(self, vocab_size, hidden_size, learning_rate=0.01):
        """
        Initialize CharRNN.
        
        Args:
            vocab_size: Number of unique characters
            hidden_size: Size of hidden state
            learning_rate: Learning rate for gradient descent
        """
        self.vocab_size = vocab_size
        self.hidden_size = hidden_size
        self.learning_rate = learning_rate
        
        # RNN weights (input to hidden, hidden to hidden)
        self.W_xh = np.random.randn(vocab_size, hidden_size) * 0.01
        self.W_hh = np.random.randn(hidden_size, hidden_size) * 0.01
        self.b_h = np.zeros(hidden_size)
        
        # Output weights (hidden to output)
        self.W_hy = np.random.randn(hidden_size, vocab_size) * 0.01
        self.b_y = np.zeros(vocab_size)
        
        print(f"Initialized CharRNN:")
        print(f"  Vocabulary size: {vocab_size}")
        print(f"  Hidden size: {hidden_size}")
        print(f"  Parameters: {self._count_parameters()}")
    
    def _count_parameters(self):
        """Count total number of parameters."""
        return (self.W_xh.size + self.W_hh.size + self.b_h.size + 
                self.W_hy.size + self.b_y.size)
    
    def forward(self, inputs):
        """
        Forward pass through the RNN.
        
        Args:
            inputs: Input sequence, shape (seq_len, vocab_size)
        
        Returns:
            outputs: Output logits, shape (seq_len, vocab_size)
            hidden_states: Hidden states, shape (seq_len, hidden_size)
        """
        seq_len = len(inputs)
        hidden_states = []
        outputs = []
        
        # Initial hidden state
        h = np.zeros(self.hidden_size)
        
        for t in range(seq_len):
            # RNN update
            h = np.tanh(inputs[t] @ self.W_xh + h @ self.W_hh + self.b_h)
            
            # Output layer
            y = h @ self.W_hy + self.b_y
            
            hidden_states.append(h.copy())
            outputs.append(y)
        
        return np.array(outputs), np.array(hidden_states)
    
    def predict(self, inputs):
        """
        Predict next character probabilities.
        
        Args:
            inputs: Input sequence, shape (seq_len, vocab_size)
        
        Returns:
            Probability distribution over vocabulary for last timestep
        """
        outputs, _ = self.forward(inputs)
        last_output = outputs[-1]
        return self._softmax(last_output)
    
    def _softmax(self, x):
        """Numerically stable softmax."""
        exp_x = np.exp(x - np.max(x))
        return exp_x / exp_x.sum()
    
# Create model
hidden_size = 10
model = CharRNN(vocab_size, hidden_size, learning_rate=0.1)

print("\n✓ CharRNN model created!")

### Test Forward Pass

Let's verify the model can process input and make predictions.

In [ ]:
# Test with first training example
test_input = inputs[0]  # "hel"
test_target = targets[0]  # expects 'l'

print(f"Input sequence: {test_input.shape}")
print(f"Target: {idx_to_char[test_target]}")

# Forward pass
outputs, hiddens = model.forward(test_input)

print(f"\nOutputs shape: {outputs.shape}")
print(f"Hidden states shape: {hiddens.shape}")

# Get prediction
probs = model.predict(test_input)
predicted_char = idx_to_char[np.argmax(probs)]

print(f"\nPrediction probabilities:")
for i, ch in enumerate(chars):
    print(f"  '{ch}': {probs[i]:.3f}")

print(f"\nPredicted character: '{predicted_char}'")
print(f"Actual target: '{idx_to_char[test_target]}'")
print("\n(Random weights, so prediction is random - we'll train it next!)")

## Step 3: Loss Function

We'll use cross-entropy loss to measure how well the model predicts the target character.

In [ ]:
def cross_entropy_loss(probs, target_idx):
    """
    Compute cross-entropy loss.
    
    Args:
        probs: Probability distribution over vocabulary
        target_idx: Index of correct character
    
    Returns:
        Loss value (scalar)
    """
    # Cross-entropy: -log(probability of correct answer)
    return -np.log(probs[target_idx] + 1e-8)  # Add small epsilon to avoid log(0)

# Test loss
test_loss = cross_entropy_loss(probs, test_target)
print(f"Loss for random prediction: {test_loss:.3f}")
print(f"Random baseline loss: {-np.log(1/vocab_size):.3f}")
print("\n(Lower is better. Random guessing gives loss ≈ {:.3f})".format(-np.log(1/vocab_size)))

## Step 4: Backpropagation Through Time (BPTT)

This is the core algorithm! We'll manually compute gradients through time.

### Mathematical Background

**Forward pass equations**:
```
h_t = tanh(W_xh @ x_t + W_hh @ h_{t-1} + b_h)
y_t = W_hy @ h_t + b_y
loss = -log(softmax(y_T)[target])
```

**Backward pass** (chain rule):
```
1. Output layer gradients:
   dL/dy_T = softmax(y_T) - target_onehot
   dL/dW_hy = h_T^T @ dL/dy_T
   dL/db_y = dL/dy_T

2. Hidden state gradients (flow backward through time):
   dL/dh_T = dL/dy_T @ W_hy^T
   dL/dh_t = dL/dh_{t+1} @ W_hh^T * tanh'(h_t)  (for t < T)

3. Weight gradients (accumulate over time):
   dL/dW_hh = sum over t: dL/dh_t^T @ h_{t-1}
   dL/dW_xh = sum over t: dL/dh_t^T @ x_t
```

Let's implement this!

In [ ]:
def backward_pass(model, inputs, target_idx, outputs, hidden_states):
    """
    Backpropagation through time (BPTT).
    
    Args:
        model: CharRNN model
        inputs: Input sequence, shape (seq_len, vocab_size)
        target_idx: Index of target character
        outputs: Forward pass outputs, shape (seq_len, vocab_size)
        hidden_states: Forward pass hidden states, shape (seq_len, hidden_size)
    
    Returns:
        Dictionary of gradients for all parameters
    """
    seq_len = len(inputs)
    
    # Initialize gradient accumulators
    dW_xh = np.zeros_like(model.W_xh)
    dW_hh = np.zeros_like(model.W_hh)
    db_h = np.zeros_like(model.b_h)
    dW_hy = np.zeros_like(model.W_hy)
    db_y = np.zeros_like(model.b_y)
    
    # ==== STEP 1: Output layer gradients ====
    # We only compute loss at the final timestep
    final_output = outputs[-1]
    probs = model._softmax(final_output)
    
    # Gradient of loss w.r.t. final output (softmax derivative)
    dy = probs.copy()
    dy[target_idx] -= 1.0  # Derivative of cross-entropy with softmax
    
    # Gradient w.r.t. output weights
    final_hidden = hidden_states[-1]
    dW_hy = np.outer(final_hidden, dy)
    db_y = dy
    
    # ==== STEP 2: Backpropagate to final hidden state ====
    dh_next = dy @ model.W_hy.T
    
    # ==== STEP 3: Backpropagate through time ====
    for t in reversed(range(seq_len)):
        # Current hidden state
        h_t = hidden_states[t]
        
        # Gradient of tanh activation
        # tanh'(x) = 1 - tanh^2(x)
        dtanh = 1 - h_t**2
        
        # Gradient before activation
        dh_raw = dh_next * dtanh
        
        # Accumulate weight gradients
        if t > 0:
            h_prev = hidden_states[t-1]
        else:
            h_prev = np.zeros(model.hidden_size)
        
        dW_hh += np.outer(h_prev, dh_raw)
        dW_xh += np.outer(inputs[t], dh_raw)
        db_h += dh_raw
        
        # Gradient for previous hidden state
        dh_next = dh_raw @ model.W_hh.T
    
    return {
        'W_xh': dW_xh,
        'W_hh': dW_hh,
        'b_h': db_h,
        'W_hy': dW_hy,
        'b_y': db_y
    }

print("✓ BPTT implementation ready!")

### Gradient Clipping

To prevent exploding gradients, we'll clip gradients to a maximum norm.

In [ ]:
def clip_gradients(gradients, max_norm=5.0):
    """
    Clip gradients to prevent explosion.
    
    Args:
        gradients: Dictionary of gradients
        max_norm: Maximum gradient norm
    
    Returns:
        Clipped gradients
    """
    # Compute total norm
    total_norm = 0
    for grad in gradients.values():
        total_norm += np.sum(grad**2)
    total_norm = np.sqrt(total_norm)
    
    # Clip if necessary
    if total_norm > max_norm:
        scale = max_norm / total_norm
        for key in gradients:
            gradients[key] *= scale
    
    return gradients, total_norm

print("✓ Gradient clipping ready!")

## Step 5: Training Loop

Now let's put it all together and train the model!

In [ ]:
def train_charrnn(model, inputs, targets, epochs=100, print_every=10):
    """
    Train the CharRNN model.
    
    Args:
        model: CharRNN model
        inputs: List of input sequences
        targets: List of target character indices
        epochs: Number of training epochs
        print_every: Print progress every N epochs
    
    Returns:
        List of losses over training
    """
    losses = []
    
    for epoch in range(epochs):
        epoch_loss = 0
        
        # Train on each example
        for input_seq, target_idx in zip(inputs, targets):
            # Forward pass
            outputs, hidden_states = model.forward(input_seq)
            
            # Compute loss
            probs = model._softmax(outputs[-1])
            loss = cross_entropy_loss(probs, target_idx)
            epoch_loss += loss
            
            # Backward pass
            grads = backward_pass(model, input_seq, target_idx, outputs, hidden_states)
            
            # Clip gradients
            grads, grad_norm = clip_gradients(grads, max_norm=5.0)
            
            # Update weights (gradient descent)
            model.W_xh -= model.learning_rate * grads['W_xh']
            model.W_hh -= model.learning_rate * grads['W_hh']
            model.b_h -= model.learning_rate * grads['b_h']
            model.W_hy -= model.learning_rate * grads['W_hy']
            model.b_y -= model.learning_rate * grads['b_y']
        
        # Average loss for epoch
        avg_loss = epoch_loss / len(inputs)
        losses.append(avg_loss)
        
        # Print progress
        if (epoch + 1) % print_every == 0:
            print(f"Epoch {epoch+1:3d}/{epochs}: Loss = {avg_loss:.4f}")
    
    return losses

print("✓ Training function ready!")
print("\nStarting training...")
print("="*50)

# Train the model!
losses = train_charrnn(model, inputs, targets, epochs=200, print_every=20)

print("="*50)
print("✓ Training complete!")

### Visualize Training Progress

In [ ]:
# Plot training loss
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.plot(losses, linewidth=2, color='blue')
plt.xlabel('Epoch', fontsize=12)
plt.ylabel('Loss', fontsize=12)
plt.title('Training Loss Over Time', fontsize=13, fontweight='bold')
plt.grid(True, alpha=0.3)
plt.axhline(y=-np.log(1/vocab_size), color='r', linestyle='--', alpha=0.5, label='Random baseline')
plt.legend()

# Smoothed loss
window = 10
if len(losses) > window:
    smoothed = np.convolve(losses, np.ones(window)/window, mode='valid')
    plt.subplot(1, 2, 2)
    plt.plot(smoothed, linewidth=2, color='green')
    plt.xlabel('Epoch', fontsize=12)
    plt.ylabel('Smoothed Loss', fontsize=12)
    plt.title(f'Smoothed Loss (window={window})', fontsize=13, fontweight='bold')
    plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"\nFinal loss: {losses[-1]:.4f}")
print(f"Random baseline: {-np.log(1/vocab_size):.4f}")
print(f"Improvement: {100 * (1 - losses[-1] / -np.log(1/vocab_size)):.1f}%")

## Step 6: Evaluation

Let's test what the model has learned!

In [ ]:
# Test on training examples
print("Testing on training data:")
print("="*50)

correct = 0
for i, (input_seq, target_idx) in enumerate(zip(inputs, targets)):
    # Get input characters
    input_chars = ''.join([onehot_to_char(input_seq[j], idx_to_char) 
                           for j in range(len(input_seq))])
    target_char = idx_to_char[target_idx]
    
    # Predict
    probs = model.predict(input_seq)
    predicted_idx = np.argmax(probs)
    predicted_char = idx_to_char[predicted_idx]
    confidence = probs[predicted_idx]
    
    # Check if correct
    is_correct = (predicted_idx == target_idx)
    if is_correct:
        correct += 1
    
    status = "✓" if is_correct else "✗"
    print(f"{status} Input: '{input_chars}' → Predicted: '{predicted_char}' (confidence: {confidence:.2f}) | Target: '{target_char}'")

accuracy = 100 * correct / len(inputs)
print("="*50)
print(f"Accuracy: {correct}/{len(inputs)} = {accuracy:.1f}%")

### Text Generation

Let's generate text by sampling from the model!

In [ ]:
def generate_text(model, seed_text, char_to_idx, idx_to_char, vocab_size, 
                  length=20, temperature=1.0):
    """
    Generate text by sampling from the model.
    
    Args:
        model: Trained CharRNN
        seed_text: Initial text to start generation
        char_to_idx: Character to index mapping
        idx_to_char: Index to character mapping
        vocab_size: Vocabulary size
        length: Number of characters to generate
        temperature: Sampling temperature (higher = more random)
    
    Returns:
        Generated text
    """
    text = seed_text
    
    for _ in range(length):
        # Take last seq_length characters as input
        input_text = text[-seq_length:]
        
        # Convert to one-hot
        input_onehot = np.array([char_to_onehot(ch, char_to_idx, vocab_size) 
                                 for ch in input_text])
        
        # Predict next character
        probs = model.predict(input_onehot)
        
        # Apply temperature
        probs = np.power(probs, 1/temperature)
        probs = probs / probs.sum()
        
        # Sample
        next_idx = np.random.choice(len(probs), p=probs)
        next_char = idx_to_char[next_idx]
        
        text += next_char
    
    return text

# Generate text
print("Text Generation Examples:")
print("="*50)

for temp in [0.5, 1.0, 1.5]:
    generated = generate_text(model, "hel", char_to_idx, idx_to_char, 
                              vocab_size, length=20, temperature=temp)
    print(f"Temperature {temp}: '{generated}'")

print("="*50)
print("\n(Temperature controls randomness: lower = more conservative)")

## Exercise 5 Summary

### What We Built

Congratulations! You just implemented a **complete language model from scratch**, including:

1. ✅ **Data preparation**: Vocabulary, one-hot encoding, training pairs
2. ✅ **Architecture**: CharRNN with input, hidden, and output layers
3. ✅ **Forward pass**: Processing sequences through the RNN
4. ✅ **Loss function**: Cross-entropy for measuring prediction quality
5. ✅ **Backpropagation Through Time**: Full BPTT implementation with gradient clipping
6. ✅ **Training**: Gradient descent optimizer
7. ✅ **Evaluation**: Testing and text generation

This is the **same core algorithm** used in:
- Early language models (2010-2015)
- Speech recognition systems
- Machine translation (before transformers)
- Text generation systems

### Key Insights

#### 1. What the Model Learned

Even on tiny data ("hello world"), the RNN learned:
- Character patterns ("hel" → "l")
- Simple sequences ("worl" → "d")
- Some structure of the text

With more data, these models can learn:
- Grammar and syntax
- Common phrases
- Longer-range dependencies (within limits)

#### 2. BPTT Challenges

**Vanishing gradients in action**:
- Even with our simple 3-character sequences, gradients decay
- For longer sequences (50+ chars), gradients become extremely small
- This is why we needed gradient clipping

**Computational cost**:
- Each training example requires:
  - Forward pass through all timesteps
  - Backward pass through all timesteps
  - Gradient accumulation and update
- For long sequences, this is SLOW

#### 3. Limitations We Encountered

**Limited Context**:
```
Our model: seq_length = 3 characters
  Can only see 3 characters back
  "hel" → "l" ✓
  But can't handle: "The cat, which was..." (requires longer memory)
```

**Small Vocabulary**:
```
Only 8 unique characters in "hello world"
Real language: 26 letters + punctuation + numbers = 50-100 characters
Full Unicode: thousands of characters!
```

**Tiny Dataset**:
```
Training text: 11 characters
Training examples: 8 pairs
Real LLMs train on billions of tokens!
```

### From CharRNN to Modern LLMs

**Our CharRNN → GPT Journey**:

1. **Bigger Data** (2013-2015):
   - Train on millions of characters/words
   - Learn more complex patterns
   - Problem: Still slow (sequential), still limited context

2. **Better Architectures** (2014-2016):
   - Use LSTMs instead of vanilla RNN
   - Stack multiple layers (deep networks)
   - Add dropout, layer normalization
   - Problem: Still sequential, still limited to ~100-500 tokens

3. **Transformers** (2017):
   - Replace RNN with self-attention
   - Parallel processing (fast!)
   - Direct connections (no vanishing gradients!)
   - Can handle 1000s of tokens
   - Result: GPT, BERT, modern LLMs!

### Why This Exercise Matters

**You now understand**:
- How language models actually work (not magic!)
- Why training is hard (vanishing gradients, optimization)
- What backpropagation does (computes gradients)
- Why we needed transformers (limitations of RNNs)

**When you learn about GPT next**:
- The attention mechanism will make sense
- You'll appreciate why it's faster
- You'll understand the architectural improvements
- You'll see it as evolution, not revolution

### Extensions (Optional Challenges)

If you want to go further, try:

1. **Longer sequences**: Increase `seq_length` to 5 or 10
   - Does it learn better?
   - Does training slow down?

2. **More data**: Use a longer training text
   - Try a paragraph or page
   - Does text generation improve?

3. **LSTM version**: Modify to use `LSTMCell` instead of vanilla RNN
   - Does it handle longer sequences better?
   - Is it more stable?

4. **Word-level model**: Instead of characters, predict words
   - Tokenize by spaces
   - Does it generate more coherent text?

**You've built something incredible!** 🎉

---

# Overall Summary & Conclusion

## What We Accomplished

Through these five exercises, you've gained **deep, hands-on understanding** of the pre-transformer era of deep learning:

### Exercise 1: RNN Experimentation ✓
**Key Learning**: Architecture matters
- Hidden size controls capacity
- tanh is preferred for RNNs (ReLU causes problems)
- Longer sequences → saturation and information loss

### Exercise 2: Vanishing Gradients ✓
**Key Learning**: The fundamental limitation
- Gradients decay exponentially with sequence length
- "Goldilocks zone" of weights is extremely narrow
- This is a mathematical inevitability, not a bug
- Motivated the invention of LSTMs and eventually transformers

### Exercise 3: LSTM Gates ✓
**Key Learning**: Adaptive memory control
- Forget, input, and output gates provide fine-grained control
- Different tasks need different gate patterns
- LSTMs partially solve vanishing gradients
- But still sequential and limited

### Exercise 4: Thinking Ahead ✓
**Key Learning**: Conceptual foundations
- Identified core problems: sequential processing, no direct access, bottlenecks
- Brainstormed solutions: attention, parallel processing, position encoding
- Prepared mentally for transformers

### Exercise 5: Character-Level Language Model ✓
**Key Learning**: End-to-end implementation
- Built complete language model with backpropagation
- Saw training dynamics first-hand
- Understood practical challenges
- Connected to modern LLMs

## Key Takeaways

### 1. RNNs Are Foundational But Limited

**What they enabled**:
- Sequential data processing
- Variable-length inputs
- Memory through hidden states
- Early successes in translation, speech, text generation

**Why they struggled**:
- Vanishing/exploding gradients
- Sequential processing (slow)
- Limited context window
- Information bottlenecks

### 2. LSTMs Improved But Didn't Solve Everything

**Improvements**:
- Gates provide adaptive memory
- Can handle ~50-100 timesteps (vs RNN's ~10-20)
- More stable training

**Remaining problems**:
- Still sequential (can't parallelize)
- Still have practical length limits
- Still have bottlenecks
- Complex architecture (hard to understand and tune)

### 3. These Limitations Motivated Transformers

**The breakthrough**: Remove recurrence entirely!
- **Attention mechanism**: Direct access to all positions
- **Parallel processing**: Process entire sequence at once
- **Positional encoding**: Explicit position information
- **Result**: GPT, BERT, ChatGPT, and modern LLMs

## Your Learning Journey

### Where You Started
- Understanding basic RNN equations
- Knowing about vanishing gradients conceptually
- Hearing about transformers and attention

### Where You Are Now
- **Deep understanding** of RNN/LSTM mechanics
- **Hands-on experience** with gradient flow and BPTT
- **Implementation skills**: Built a language model from scratch
- **Conceptual foundation**: Ready for transformers

## What's Next?

You're now perfectly prepared for the next notebooks:

### Next: `02_attention_mechanism.ipynb`

You'll learn:
- **How attention works**: Query, Key, Value mechanism
- **Self-attention**: How words attend to each other
- **Multi-head attention**: Parallel attention in different subspaces
- **Implementation**: Build attention from scratch

### Then: `03_transformer_architecture.ipynb`

You'll learn:
- **Complete transformer**: Encoder and decoder
- **Positional encoding**: How position information is added
- **Layer normalization**: Stabilizing deep networks
- **Residual connections**: Enabling deep stacks

### Eventually: Building GPT

You'll:
- Understand decoder-only transformers
- Learn about tokenization
- Train a tiny GPT model
- Generate text with transformers

## Final Thoughts

### You Understand the "Why" Now

When someone says "transformers are better than RNNs," you don't just nod - you **understand**:
- Why: parallel processing, direct connections, no vanishing gradients
- How: attention mechanism, positional encoding
- Trade-offs: different strengths and weaknesses

### You Can Build Things

You've implemented:
- RNNs and LSTMs from scratch
- Backpropagation through time
- A complete language model

These are **real skills** that transfer to:
- Reading research papers
- Understanding model architectures
- Debugging training issues
- Building custom models

### You're Part of the Story

The journey from RNNs → LSTMs → Transformers is:
- Not just history
- Not just trivia
- **The foundation of modern AI**

Every time you use:
- ChatGPT
- Google Translate
- Voice assistants
- Code completion

...you're using ideas built on what you just learned!

## Congratulations! 🎉

You've completed the pre-transformer era! You have:
- ✅ Solid understanding of RNNs and LSTMs
- ✅ Hands-on implementation experience
- ✅ Deep knowledge of why transformers were needed
- ✅ Strong foundation for learning attention

**You're ready for the attention mechanism!**

See you in the next notebook! 🚀